## Usage

Run the `report_historic_ocurrences` to either create or update OcurrenceReports for the end of every day since Aug 8 2020 til today.

In [1]:
import arrow
from datetime import datetime
from typing import List

from apps.patterns.git import repo_at
from apps.patterns.models import OcurrenceReport
from apps.patterns.analyzer import analyze_repo_with_pattern
from apps.utils import to_arrow

def get_end_of_day(arrow_date, days_ago=1, tz="America/Santiago"):
    return arrow_date.shift(days=-days_ago).replace(hour=23, minute=59, second=59, tzinfo=tz)

def report_historic_ocurrences(study_app: str, ignore_paths: List[str] = []):
    now = arrow.now()
    max_datetime = arrow.Arrow(year=2021, month=4, day=22
                               , hour=0, mimute=0, second=0)
    days_left_to_analyze = (now - max_datetime).days
    last_of_days_commits = []
    with repo_at(commit=None, force_delete=False, always_pull=True) as repo:
        current_day = 1
        skip_commits = 0
        max_page = 1000
        commits = repo.iter_commits("devel", max_count=max_page, skip=skip_commits)
        while current_day != days_left_to_analyze:
            critic_moment = get_end_of_day(now, days_ago=current_day)

            last_day_commit_found = False
            while not last_day_commit_found:
                try:
                    commit = next(commits)
                except StopIteration:
                    skip_commits += max_page
                    commits = repo.iter_commits("devel", max_count=max_page, skip=skip_commits)
                    commit = next(commits)
 
                commit_datetime = to_arrow(commit.committed_datetime)
                if commit_datetime < critic_moment:
                    last_of_days_commits.append(commit)
                    print(f"added new study commit for {commit_datetime}")
                    last_day_commit_found = True

            current_day += 1

    for commit in last_of_days_commits:
        try:
            report = OcurrenceReport.objects.filter(commit=commit.hexsha, app_name=study_app).get()
            report.last_of_day = True
            report.save()
            print(f"Found stored commit {commit.hexsha}")
        except OcurrenceReport.DoesNotExist:
            print(f"Moving repo and analyzing {commit.hexsha}")
            with repo_at(commit=commit.hexsha, force_delete=False, always_pull=True) as repo:
                hits = analyze_repo_with_pattern(study_app=study_app, pattern=f"apps.{study_app}.models", ignore_paths=ignore_paths)
                payload = dict(
                    app_name=study_app,
                    ocurrences=hits,
                    commit=commit.hexsha,
                    commited_epoch=to_arrow(commit.committed_datetime).timestamp,
                    last_of_day=True
                )
                OcurrenceReport.objects.create(**payload)

                
print("aaaffds")
## Study Catalog
report_historic_ocurrences(study_app="catalog", ignore_paths=["apps/catalog", "apps/promotions", "apps/stores"])

## Study promotions
report_historic_ocurrences(study_app="promotions", ignore_paths=["apps/catalog", "apps/promotions", "apps/stores"])

## Study stores
report_historic_ocurrences(study_app="stores", ignore_paths=["apps/catalog", "apps/promotions", "apps/stores"])

## Study Orders
#report_historic_ocurrences(study_app="orders", ignore_paths=["apps/orders"])

aaaffds
added new study commit for 2021-05-02T13:28:25-04:00
added new study commit for 2021-05-01T21:28:18-04:00
added new study commit for 2021-04-30T21:57:35-04:00
added new study commit for 2021-04-29T23:35:15-04:00
added new study commit for 2021-04-28T23:26:44-04:00
added new study commit for 2021-04-27T22:42:55-04:00
added new study commit for 2021-04-26T23:12:18-04:00
added new study commit for 2021-04-24T15:50:08-04:00
added new study commit for 2021-04-24T14:46:11-04:00
added new study commit for 2021-04-23T20:39:05-04:00
Moving repo and analyzing 2f41346cd6d165ad3f5e41ab2e4f574611aa438e
Moving repo and analyzing 0213cf9aeda1c061a3eb1b66742520c12fa4129b
Moving repo and analyzing e1720aaacaf1ce8631c91bfe3cb6ade67c510eab
Moving repo and analyzing a337b9303f45ff84b3e40ee531ec7a26fa08d0be
Moving repo and analyzing 0dfc5f958ecce5e2340e337f1951f814a23bd8f0
Moving repo and analyzing 2cb5f7320a4c5535f2df64acc9a862024bf4a721
Moving repo and analyzing 854ab9808d61fc516597800a870f15edfb

In [4]:
import arrow
for app in ["catalog", "promotions", "stores", "orders"]:
    all_reports = OcurrenceReport.objects.filter(last_of_day=True, app_name=app).order_by('commited_epoch').values_list('_total', 'commited_epoch')
    print()
    print("======")
    print(app)
    print("======")
    print()
    for tot, epoch in all_reports:
        print(arrow.get(epoch).format('YYYY-MM-DD'))
    for tot, epoch in all_reports:
        print(tot)


catalog

2020-08-17
2020-08-18
2020-08-19
2020-08-20
2020-08-21
2020-08-22
2020-08-23
2020-08-24
2020-08-25
2020-08-26
2020-08-27
2020-08-28
2020-08-29
2020-08-31
2020-09-01
2020-09-02
2020-09-03
2020-09-04
2020-09-05
2020-09-07
2020-09-08
2020-09-09
2020-09-10
2020-09-11
2020-09-12
2020-09-13
2020-09-15
2020-09-16
2020-09-17
2020-09-18
2020-09-20
2020-09-21
2020-09-22
2020-09-23
2020-09-24
2020-09-25
2020-09-26
2020-09-27
2020-09-29
2020-09-30
2020-10-01
2020-10-02
2020-10-03
2020-10-04
2020-10-06
2020-10-07
2020-10-08
2020-10-09
2020-10-10
2020-10-11
2020-10-13
2020-10-14
2020-10-15
2020-10-16
2020-10-17
2020-10-18
2020-10-19
2020-10-20
2020-10-21
2020-10-22
2020-10-23
2020-10-24
2020-10-25
2020-10-26
2020-10-27
2020-10-28
2020-10-29
2020-10-30
2020-10-31
2020-11-01
2020-11-03
2020-11-04
2020-11-05
2020-11-06
2020-11-07
2020-11-09
2020-11-10
2020-11-11
2020-11-12
2020-11-13
2020-11-14
2020-11-16
2020-11-17
2020-11-18
2020-11-19
2020-11-20
2020-11-21
2020-11-22
2020-11-23
2020-11-24


615
615
617
617
611
611
617
612
611
611
606
606
607
608
608
608
608
608
598
595
595
584
584
584
584
581
582
579
579
579
579
579
579
579
579
579
583
583
583
583
583
583
582
581
581
579


In [5]:
import arrow
domains=dict(
    catalog=dict(apps=["catalog", "promotions", "stores"]),
    orders=dict(apps=["orders", "orders_archive", "orders_archive_viewer" "assigner", "dispatcher"])
)
chosen_domain = "catalog"
domain_data = domains[chosen_domain]
all_reports = OcurrenceReport.objects.aggregate_last_of_day_from_apps(apps=["catalog", "stores", "promotions"])
for date, count in all_reports.items():
    print(date)
for date, count in all_reports.items():
    print(count)

2020-08-17
2020-08-18
2020-08-19
2020-08-20
2020-08-21
2020-08-22
2020-08-23
2020-08-24
2020-08-25
2020-08-26
2020-08-27
2020-08-28
2020-08-29
2020-08-31
2020-09-01
2020-09-02
2020-09-03
2020-09-04
2020-09-05
2020-09-07
2020-09-08
2020-09-09
2020-09-10
2020-09-11
2020-09-12
2020-09-13
2020-09-15
2020-09-16
2020-09-17
2020-09-18
2020-09-20
2020-09-21
2020-09-22
2020-09-23
2020-09-24
2020-09-25
2020-09-26
2020-09-27
2020-09-29
2020-09-30
2020-10-01
2020-10-02
2020-10-03
2020-10-04
2020-10-06
2020-10-07
2020-10-08
2020-10-09
2020-10-10
2020-10-11
2020-10-13
2020-10-14
2020-10-15
2020-10-16
2020-10-17
2020-10-18
2020-10-19
2020-10-20
2020-10-21
2020-10-22
2020-10-23
2020-10-24
2020-10-25
2020-10-26
2020-10-27
2020-10-28
2020-10-29
2020-10-30
2020-10-31
2020-11-01
2020-11-03
2020-11-04
2020-11-05
2020-11-06
2020-11-07
2020-11-09
2020-11-10
2020-11-11
2020-11-12
2020-11-13
2020-11-14
2020-11-16
2020-11-17
2020-11-18
2020-11-19
2020-11-20
2020-11-21
2020-11-22
2020-11-23
2020-11-24
2020-11-25

In [5]:
from apps.patterns.models import OcurrenceReport
rep = OcurrenceReport.objects.filter(last_of_day=True, app_name="catalog").order_by("-commited_epoch").values_list('_total', 'commited_epoch').first()
#rep.total

print(rep)
print(arrow.get(rep[1]).format('YYYY-MM-DD'))
print(rep[0])

(132, 1612144890)
2021-02-01
132


# Clean ocurrences

Sometimes more than one OcurrenceReport with last_of_day=True is created for the same day. We should only keep the "latest" one

In [3]:
from apps.patterns.models import OcurrenceReport

for app_name in ("catalog", "promotions", "stores", "orders"):
    reports = OcurrenceReport.objects.filter(last_of_day=True, app_name=app_name)
    acc = {}

    for report in reports:
        report_date = arrow.get(report.commited_epoch).date()
        if report_date in acc:
            element = acc[report_date]
            element.append((report.id, report.commited_epoch))
            acc[report_date] = element
        else:
            acc[report_date] = [(report.id, report.commited_epoch)]

    ids_to_delete = []
    for date, metadata in acc.items():
        if len(metadata) > 1:
            ordered_metadata = sorted(metadata, key=lambda x: x[1], reverse=True)
            all_but_first = ordered_metadata[1:]
            ids_to_delete.extend(list(map(lambda x: x[0], all_but_first)))

    print(f"{app_name}: {ids_to_delete}")
    print(OcurrenceReport.objects.filter(id__in=ids_to_delete).delete())

catalog: [2163, 2164, 2157]
(3, {'patterns.OcurrenceReport': 3})
promotions: [2172, 2173, 2166]
(3, {'patterns.OcurrenceReport': 3})
stores: [2181, 2182, 2175]
(3, {'patterns.OcurrenceReport': 3})
orders: []
(0, {'patterns.OcurrenceReport': 0})


In [15]:
from apps.patterns.models import OcurrenceReport

ocr = OcurrenceReport.objects.get(id=1252)
ocr.commit

'ccb406231e95aa81543f61563dd74b8595b9b573'